In [1]:
import requests
from last_fm_secrets import *
import time
import pandas as pd

In [2]:
class Function_Dictionary_Class:
    user_agent = "Halcyon"
    api_key    = api_key
    secret     = secret
    api_root_url = "http://ws.audioscrobbler.com/2.0"
    method       = "user.getRecentTracks"
    user         = username
    json         = "json"
    root_key     = "recenttracks"
    attributes   = "@attr"
    tracks       = "track"
    mb_id        = "mbid"
    artist       = "artist"
    name         = "name"
    album        = "album"
    date         = "date"
    uts          = "uts"
    text         = "#text"

    class DataFrame_Columns_Class:
        track_id    = "track_id"
        artist_id   = "artist_id"
        album_id    = "album_id"
        track_name  = "track_name"
        artist_name = "artist_name"
        album_name  = "album_name"
        date        = "date"
    
    def __init__(self):
        self.df_columns = self.DataFrame_Columns_Class()

func_dict = Function_Dictionary_Class()

headers = {
    "user-agent": func_dict.user_agent
}

In [3]:
def retrieve_page(user_name, page):
    payload = {
        "api_key": func_dict.api_key,
        "method": func_dict.method,
        "format": func_dict.json,
        "limit": 200,
        "user": user_name,
        "extended": 1,
        "page": page
    }

    r = requests.get(func_dict.api_root_url, headers=headers, params=payload)
    retrieved_json = r.json()
    attributes = retrieved_json[func_dict.root_key][func_dict.attributes]
    if page > int(attributes['totalPages']):
        raise ValueError("Exceeded total number of pages")

    return retrieved_json[func_dict.root_key][func_dict.tracks]

def parse_page(page_data):
    
    temp_list = []

    for entry in page_data:
        temp_list.append({
            func_dict.df_columns.track_name:    entry[func_dict.name],
            func_dict.df_columns.track_id:      entry[func_dict.mb_id],
            func_dict.df_columns.artist_name:   entry[func_dict.artist][func_dict.name],
            func_dict.df_columns.artist_id:     entry[func_dict.artist][func_dict.mb_id],
            func_dict.df_columns.album_name:    entry[func_dict.album][func_dict.text],
            func_dict.df_columns.album_id:      entry[func_dict.album][func_dict.mb_id],
            func_dict.df_columns.date:          entry[func_dict.date][func_dict.uts]
        })
    
    return_df = pd.DataFrame(temp_list)
    return_df[func_dict.df_columns.date] = pd.to_datetime(return_df[func_dict.df_columns.date], unit="s")
    return return_df

In [15]:
def retrieve_all_data(user_name):

    #Retrieve first page, just for attribute examination
    payload = {
        "api_key": func_dict.api_key,
        "method": func_dict.method,
        "format": func_dict.json,
        "limit": 200,
        "user": user_name,
        "extended": 1,
    }
    r = requests.get(func_dict.api_root_url, headers=headers, params=payload)
    retrieved_json = r.json()
    attributes = retrieved_json[func_dict.root_key][func_dict.attributes]
    total_pages = int(attributes['totalPages'])

    list_of_dataframes = []
    for i in range(total_pages):
        print(f"\rPage {i + 1} of {total_pages}", end="")
        while True:
            try:
                page_data = retrieve_page(user_name, i + 1)
                break
            except KeyError:
                time.sleep(0.1)
                continue

        list_of_dataframes.append(parse_page(page_data))

        time.sleep(0.1)
    
    return pd.concat(list_of_dataframes, ignore_index=True)

In [16]:
df = retrieve_all_data("rawrspoon")

Page 642 of 642

In [17]:
df

,track_name,track_id,artist_name,artist_id,album_name,album_id,date
0,Can't Help,2e585c77-48e8-420d-9649-28c188e9fc0f,Parachute,,Can't Help,75fd3dcb-62e2-4a86-b479-c14903d5f57a,2020-12-17 00:55:08
1,Canned Heat,045ef838-e886-4155-ad57-116bda32b97b,Jamiroquai,,Synkronized,2ab9c2ba-8026-4f80-ae12-56450b1165fb,2020-12-17 00:49:36
2,Beach Bones (Feat. Ryan Ross),,More Amor,,Beach Bones (feat. Ryan Ross),,2020-12-17 00:46:10
3,Push Push (Lady Lightning),63aea131-a5f2-41c3-b689-aad85f9c46ec,Bang Camaro,,Bang Camaro,2defe6da-a0d0-419d-83a7-1709e1f3fc62,2020-12-17 00:41:28
4,Procrastinating,4f90aefc-acda-3807-8ba7-4bb829e1936e,Stellar Kart,,Life Is Good: The Best of Stellar Kart,,2020-12-17 00:32:43
...,...,...,...,...,...,...,...
128343,Cupid Shuffle,2907feb9-c3b0-4b03-94af-ba2b38e7ae93,Cupid,,Time For A Change,4bbbe6a2-64a9-4550-8c1d-c9590591b4d7,2013-09-07 11:05:10
128344,Out of My Head,21195bf8-14f7-3601-8064-2026d46ffc28,Theory of a Deadman,,The Truth Is...,4f1279aa-98af-43dd-9b90-8d0d3d00c12d,2013-09-07 11:02:49
128345,Painkiller,039cd1bb-8fc9-3fcc-a902-0af24a1e05cc,Judas Priest,,Judas Priest - The Essential,,2013-09-07 10:56:31
128346,The Anthem,17e96d04-f759-3125-865e-ec4b19f81ba8,Good Charlotte,,The Anthem,29b20ee4-d4df-40d6-98da-7c1d229ee1da,2013-09-07 10:53:36


In [23]:
df.to_hdf("listener_df.h5", key="df", mode="w")

In [24]:
pd.read_hdf("listener_df.h5", key="df")

,track_name,track_id,artist_name,artist_id,album_name,album_id,date
0,Can't Help,2e585c77-48e8-420d-9649-28c188e9fc0f,Parachute,,Can't Help,75fd3dcb-62e2-4a86-b479-c14903d5f57a,2020-12-17 00:55:08
1,Canned Heat,045ef838-e886-4155-ad57-116bda32b97b,Jamiroquai,,Synkronized,2ab9c2ba-8026-4f80-ae12-56450b1165fb,2020-12-17 00:49:36
2,Beach Bones (Feat. Ryan Ross),,More Amor,,Beach Bones (feat. Ryan Ross),,2020-12-17 00:46:10
3,Push Push (Lady Lightning),63aea131-a5f2-41c3-b689-aad85f9c46ec,Bang Camaro,,Bang Camaro,2defe6da-a0d0-419d-83a7-1709e1f3fc62,2020-12-17 00:41:28
4,Procrastinating,4f90aefc-acda-3807-8ba7-4bb829e1936e,Stellar Kart,,Life Is Good: The Best of Stellar Kart,,2020-12-17 00:32:43
...,...,...,...,...,...,...,...
128343,Cupid Shuffle,2907feb9-c3b0-4b03-94af-ba2b38e7ae93,Cupid,,Time For A Change,4bbbe6a2-64a9-4550-8c1d-c9590591b4d7,2013-09-07 11:05:10
128344,Out of My Head,21195bf8-14f7-3601-8064-2026d46ffc28,Theory of a Deadman,,The Truth Is...,4f1279aa-98af-43dd-9b90-8d0d3d00c12d,2013-09-07 11:02:49
128345,Painkiller,039cd1bb-8fc9-3fcc-a902-0af24a1e05cc,Judas Priest,,Judas Priest - The Essential,,2013-09-07 10:56:31
128346,The Anthem,17e96d04-f759-3125-865e-ec4b19f81ba8,Good Charlotte,,The Anthem,29b20ee4-d4df-40d6-98da-7c1d229ee1da,2013-09-07 10:53:36
